In [15]:
import sympy as sp

x = sp.Symbol("x", positive = True)
y = sp.Symbol("y", positive = True)
z = sp.Symbol("z", positive = True)
l1 = sp.Symbol("l1", positive = True)
l2 = sp.Symbol("l2", positive = True)
l3 = sp.Symbol("l3", positive = True)
l4 = sp.Symbol("l4", positive = True)
q1 = sp.Symbol("q1", positive = True)
q2 = sp.Symbol("q2", positive = True)
q3 = sp.Symbol("q3", positive = True)
q4 = sp.Symbol("q4", positive = True)

phi = sp.Symbol("phi", positive = True)
pie = sp.Symbol("pi", positive = True)

In [16]:
def translateX(x):
    return sp.Matrix([[1, 0, 0, x], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

def translateZ(z):
    return sp.Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, z], [0, 0, 0, 1]])

def rotateX(theta):
    return sp.Matrix([[1, 0, 0, 0], [0, sp.cos(theta), -sp.sin(theta), 0], [0, sp.sin(theta), sp.cos(theta), 0], [0, 0, 0, 1]])

def rotateZ(theta):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, 0], [sp.sin(theta), sp.cos(theta), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

def rotateY(theta):
    return sp.Matrix([[sp.cos(theta), 0, sp.sin(theta), 0], [0, 1, 0, 0], [-sp.sin(theta), 0, sp.cos(theta), 0], [0, 0, 0, 1]])

In [17]:
DH = sp.Matrix([[0, 0, l1, 0, 0],
                [0, sp.pi/2, 0, sp.pi/2, 0],
                [l2, 0, 0, 0, 0],])
DH

Matrix([
[ 0,    0, l1,    0, 0],
[ 0, pi/2,  0, pi/2, 0],
[l2,    0,  0,    0, 0]])

In [18]:
H  = sp.Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
q = [q1,q2,q3,q4]
for i in range(DH.shape[0]):
    if DH[i, 4] == 0:# Revolute joint
        Hj = rotateZ(q[i])
    elif DH[i, 4] == 1: # Prismatic joint
        Hj = translateZ(q[i])
    else: # Fixed joint
        Hj = sp.Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    H = H * translateX(DH[i, 0]) * rotateX(DH[i, 1]) * translateZ(DH[i, 2]) * rotateZ(DH[i, 3]) * Hj
H = sp.trigsimp(H)
H


Matrix([
[-sin(q2 + q3)*cos(q1), -cos(q1)*cos(q2 + q3),  sin(q1), -l2*sin(q2)*cos(q1)],
[-sin(q1)*sin(q2 + q3), -sin(q1)*cos(q2 + q3), -cos(q1), -l2*sin(q1)*sin(q2)],
[         cos(q2 + q3),         -sin(q2 + q3),        0,     l1 + l2*cos(q2)],
[                    0,                     0,        0,                   1]])

In [19]:
H3e = sp.Matrix([[0, 0, 1, 0],
                 [-1, 0, 0, -l3],
                 [0, -1, 0, 0],
                 [0, 0, 0, 1]])
He = H * H3e
He = sp.trigsimp(He)
He

Matrix([
[cos(q1)*cos(q2 + q3), -sin(q1), -sin(q2 + q3)*cos(q1), (-l2*sin(q2) + l3*cos(q2 + q3))*cos(q1)],
[sin(q1)*cos(q2 + q3),  cos(q1), -sin(q1)*sin(q2 + q3), (-l2*sin(q2) + l3*cos(q2 + q3))*sin(q1)],
[        sin(q2 + q3),        0,          cos(q2 + q3),       l1 + l2*cos(q2) + l3*sin(q2 + q3)],
[                   0,        0,                     0,                                       1]])

In [20]:
P = sp.trigsimp(He.col(3).row([0,1,2]))
eqP = sp.Eq(sp.Matrix([x, y, z]),P)
eqP

Eq(Matrix([
[x],
[y],
[z]]), Matrix([
[(-l2*sin(q2) + l3*cos(q2 + q3))*cos(q1)],
[(-l2*sin(q2) + l3*cos(q2 + q3))*sin(q1)],
[      l1 + l2*cos(q2) + l3*sin(q2 + q3)]]))

In [21]:
r1 = sp.Symbol("r1", positive = True)
A = sp.Matrix([[z-l1],[r1*sp.sqrt(x**2+y**2)]])
A

Matrix([
[             -l1 + z],
[r1*sqrt(x**2 + y**2)]])

In [22]:
B = sp.Matrix([[l2+l3*sp.sin(q3),l3*sp.cos(q3)],[l3*sp.cos(q3),-l2-l3*sp.sin(q3)]])
B

Matrix([
[l2 + l3*sin(q3),       l3*cos(q3)],
[     l3*cos(q3), -l2 - l3*sin(q3)]])

In [23]:
Binv = B.inv()
B

Matrix([
[l2 + l3*sin(q3),       l3*cos(q3)],
[     l3*cos(q3), -l2 - l3*sin(q3)]])

In [24]:
Ans = Binv*A
Ans = sp.trigsimp(Ans)
Ans

Matrix([
[               (-l1*l2 - l1*l3*sin(q3) + l2*z + l3*r1*sqrt(x**2 + y**2)*cos(q3) + l3*z*sin(q3))/(l2**2 + 2*l2*l3*sin(q3) + l3**2)],
[l3*(-l1 + z)*cos(q3)/(l2**2 + 2*l2*l3*sin(q3) + l3**2) - r1*(l2 + l3*sin(q3))*sqrt(x**2 + y**2)/(l2**2 + 2*l2*l3*sin(q3) + l3**2)]])